# Linkit Challenge WS 2022/23 Hand Gesture Detection - Team X

## 1. Setup


First, you usually create a virtual environment where you can keep track of all package versions.
Create the environment with the following command in Terminal (Mac)/ CMD (Windows)

python3 -m venv venv


Then activate it with
- in CMD: venv\Scripts\activate
- in Terminal: . venv/bin/activate

In [1]:
# only run once to setup your environment
first_run = True

if first_run:
    # install reqiurements.txt
    !pip3 install -r requirements.txt
    # clone the git repo of ultralytics/yolov5
    !git clone https://github.com/ultralytics/yolov5.git
    # install dependencies for yolo
    !pip3 install -r yolov5/requirements.txt
    # or from website
    #!pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

  Using cached GitPython-3.1.29-py3-none-any.whl (182 kB)
  Using cached setuptools-65.5.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.8.0
    Uninstalling setuptools-67.8.0:
      Successfully uninstalled setuptools-67.8.0
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.31
    Uninstalling GitPython-3.1.31:
      Successfully uninstalled GitPython-3.1.31

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Cloning into 'yolov5'...
remote: Enumerating objects: 15935, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 15935 (delta 65), reused 87 (delta 58), pack-reused 15831
Receiving objects: 100% (15935/15935), 14.53 MiB | 4.99 MiB/s, done.
Resolving deltas: 100% (10939/10939), done.
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached se

#### 1.1. Import Packages

In [2]:
import pytorch_lightning as pl # PyTorch Lightning for easier training and evaluation of models
import torch # PyTorch
import cv2  # OpenCV for image processing
import matplotlib.pyplot as plt # for plotting
import matplotlib.patches as patches  # for plotting bounding boxes
%matplotlib inline
import uuid   # Unique identifier
import os # File system operations
import time # Time operations


/Users/justusthomsen/Documents/LinkIT/Challenges/Expert Challenge/linkit_hand_gesture_recognition/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Annotation

## 2.1 Capture Images

Select the number of how many images you want to take per class and choose the path were these images will be stored

In [4]:
labels = ['rock', 'paper', 'scissor']
number_imgs = 1
IMAGES_PATH = os.path.join('datasets', 'gestures', 'images')
LABELS_PATH = os.path.join('datasets', 'gestures', 'labels')

Create the directory (works for all operation systems)

In [18]:
def create_dir(path):
    if not os.path.exists(path):
        !mkdir {path}


In [31]:
for structure in [IMAGES_PATH,LABELS_PATH]:
    create_dir(structure)
    # create subfolder
    for folder in ['test-dev','train','val']:
        create_dir(os.path.join(structure, folder))

Capture the images! You can adjust the time between the different frames to have more time of switching between your hand gestures.

In [33]:
for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(7) #time between gestures
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,'train',label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2) #time between frames

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Go to https://www.makesense.ai and label your data there. First create the label rock, then paper and then scissor to ensure that they are labeled as 0, 1, 2.

**Export the labels in yolo format**

## 3. Training

#### 3.2. Load and Test Pretrained Model
In Computer Vision, we usually use pretrained Models, to reduce the number of samples required for training. In this case, we use a pretrained YOLOv5 model, which was trained on the COCO dataset. The COCO dataset contains 80 different classes, which are not relevant for our task. For now, let's just test the model on a random image.

#### Tasks:
_**Task 3.1:**_ Can you find better models for our task?
_**Task 3.2:**_ Can you find better pretrained weights for our task?
_Note:_ You can find possible models and weights on:
1. huggingface [Model Hub](https://huggingface.co/models?pipeline_tag=object-detection&sort=downloads).
2. pytorch [Model Zoo](https://pytorch.org/docs/stable/torchvision/models.html).
3. pytorch [Model Hub](https://pytorch.org/hub/).


In [32]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /Users/justusthomsen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-1 Python-3.10.10 torch-1.13.0 CPU

100%|██████████| 14.1M/14.1M [00:05<00:00, 2.61MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


We Load  an Image and run the model on it. The model returns a list of bounding boxes, with their corresponding class and confidence score.

In [34]:
# Load image
img = cv2.imread('datasets/example/zidane.png')

In [35]:
results = model(img)

In [36]:
print(results)

image 1/1: 720x1280 2 persons, 1 tie
Speed: 3.1ms pre-process, 61.2ms inference, 1.4ms NMS per image at shape (1, 3, 384, 640)


In [10]:
# Display image and Convert to RGB, display labels and bounding boxes from the results with cv2
fig, ax = plt.subplots()
ax.imshow(img[:,:,::-1])

# Draw bounding boxes and labels of detections
for *rect,  conf, name, cls   in results.pandas().xyxy[0].values:
    x1, y1, x2, y2 = rect
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.text(x1, y1, f'{cls} {conf:.2f}', fontsize=12, c='white')
plt.show()

#### 3.3 Adapt the Model for our Task
We need to adapt the model for our task. Therefore, we need to remove the last layer of the model, and replace it with a new layer, which only contains 3 classes (one for each hand gesture).
Since we use the [yolov5 implementation of ultralytics](https://github.com/ultralytics/yolov5), we can use their provided training script to train our model.


First we define our 3 label names, and the path to the images and labels.

This is done in the dataset.yaml file.
Here we have to define the 3 classes, and the path to the images and labels.
Make sure to use the correct mapping between classID and label
1. 0 -> "rock"
2. 1 -> "paper"
3. 2 -> "scissors"

Now we train our model.
we use the dataset.yml file to define the path to the images and labels, and the number of classes.

For different Parameter Configurations refer to:
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

### Monitoring
It is important to monitor the training process to ensure that the model is training properly.
To do so, we recommend [Weights and Biases](https://wandb.ai/) (or [tensorboard](https://www.tensorflow.org/tensorboard)).
Both tools keep track of the training process and automatically log the results.
#### Weights and Biases
1. Create an account on [Weights and Biases](https://wandb.ai/)
2. Install the wandb package `pip install wandb`
3. Login to your account `wandb login`
4. Run the training script with the `--project` flag `python train.py --project <project_name>`
5. Go to your [Weights and Biases](https://wandb.ai/) dashboard to view the results

For more information on the YOLOV5 integration with Weights and Biases, refer to [here](https://docs.wandb.ai/guides/integrations/yolov5)


In [33]:
# Train YOLOv5s on COCO128 for 3 epochs
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 3 --data ../dataset.yaml --weights yolov5s.pt --cache --workers 0 --save-period 1

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: justus-thomsen (linkit). Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=../dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=0, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/Users/justusthomsen/Documents/LinkIT/Challenges/Expert': No 

In [45]:
!cd yolov5 && python3 detect.py --weights runs/train/exp/weights/last.pt --img 640 --conf 0.01 --source ../datasets/gestures/images/val

detect: weights=['runs/train/exp/weights/last.pt'], source=../datasets/gestures/images/val, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.01, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/Users/justusthomsen/Documents/LinkIT/Challenges/Expert': No such file or directory
YOLOv5 🚀 2023-6-1 Python-3.10.10 torch-1.13.0 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/6 /Users/justusthomsen/Documents/LinkIT/Challenges/Expert Challenge/linkit_hand_gesture_recognition/datasets/gestures/images/val/paper.c9547264-fe04-11ed-a540-161b77bd6551.jpg: 384x640 10 rocks, 10 papers, 1 scissor, 45.6ms
image 2/6 /Users/justusthomsen/

## 4. Inference
Now that we have trained our model, we can use it to detect hand gestures in images.
We can use the `detect.py` script to detect hand gestures in images.

1. The `--source 0` argument specifies that we want to use the webcam as input.
2. The `--weights path.pt` argument specifies the path to the weights of the model.
3. The `--conf 0.X` argument specifies the confidence threshold.

The confidence threshold determines the minimum confidence score for a bounding box to be considered as a detection. If the confidence score is below the threshold, the bounding box will be ignored. This is useful to filter out false positives.

In [40]:
!cd yolov5 && python3 detect.py --source 0 --weights runs/train/exp/weights/last.pt --conf 0.4

detect: weights=['../yolov5s.pt'], source=0, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/Users/justusthomsen/Documents/LinkIT/Challenges/Expert': No such file or directory
YOLOv5 🚀 2023-6-1 Python-3.10.10 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
1/1: 0...  Success (inf frames 1920x1080 at 30.00 FPS)

0: 384x640 (no detections), 53.1ms
0: 384x640 (no detections), 50.0ms
0: 384x640 (no detections), 49.4ms
0: 384x640 (no detections), 49.6ms
0: 384x640 (no detections), 48.3ms
0: 384x640 (no detections), 51.1ms
0: 384x640 (no detections), 50.3ms
0: 384x640 (no detect